In [ ]:
from optimus import Optimus
op = Optimus()

In [ ]:
op.spark

In [ ]:
op.sc

## Exploratory Data Analysis

In [ ]:
df = op.load.csv("data/train.csv")

In [ ]:
df.table()

In [ ]:
op.profiler.missing_values(df, "*")

In [ ]:
op.profiler.run(df, "*",infer=True)

In [ ]:
df.plot.box("ApplicantIncome")

In [ ]:
df.plot.hist("LoanAmount", buckets=100)

## Data cleansing

In [ ]:
from pyspark.sql.functions import isnull, when, count, col

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

In [ ]:
df.where(col("Gender").isNull()).table()

In [ ]:
[row.Loan_ID for row in df.where(col("Gender").isNull()).select("Loan_ID").collect()]

In [ ]:
df_imputed = df.cols.impute("Gender", data_type="categorical")

In [ ]:
df_imputed.where(col("Loan_ID").isin(['LP001050',
 'LP001448',
 'LP001585',
 'LP001644',
 'LP002024',
 'LP002103',
 'LP002478',
 'LP002501',
 'LP002530',
 'LP002625',
 'LP002872',
 'LP002925',
 'LP002933'])).table()

## Data preparation

In [ ]:
df = df.cols.impute("LoanAmount",data_type="continuous")

In [ ]:
df = df.cols.fill_na("Self_Employed", "No")
df = df.cols.fill_na("*", 0)

In [ ]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

In [ ]:
from optimus.ml import feature as fe

In [ ]:
fe.string_to_index?

In [ ]:
df_idx = fe.string_to_index(df, ['Gender','Married','Dependents','Education','Self_Employed','Property_Area'])

In [ ]:
df_idx.table(10)

## Modeling

In [ ]:
columns = ['Loan_Status','Credit_History','Gender_index','Married_index','Education_index']

In [ ]:
df_model, dt_model = op.ml.decision_tree(df_idx, columns, "Loan_Status")

In [ ]:
df_model.select("label","features","probability","prediction").show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(df_model)

In [ ]:
print(f'Accuracy = {accuracy}')